# OSMOSIS Winter

This notebook runs [GOTM](https://gotm.net/) with initial conditions and surface forcing during the winter months (Sep. 22, 2012 - Dec. 5, 2012) of the Ocean Surface Mixing, Ocean Submesoscale Interaction Study in the northeast Atlantic (OSMOSIS, 48.7$^\circ$N, 16.2$^\circ$W; [Damerell et al., 2016](https://doi.org/10.1002/2015JC011423)). See Fig. 2 of [Li et al., 2019](https://doi.org/10.1029/2019MS001810).

In [ ]:
import sys
import copy
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../../../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [ ]:
m = Model(name='OSMOSIS-Winter', environ='../../.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [ ]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

## Build the model

In [ ]:
%%time
m.build()

## Configuration
Initialize the GOTM configuration

In [ ]:
cfg = m.init_config()

Update the configuration

In [ ]:
# setup
title = 'OSMOSIS - Winter'
nlev = 200
cfg['title'] = title
cfg['location']['name'] = 'OSMOSIS'
cfg['location']['latitude'] = 48.7
cfg['location']['longitude'] = -16.2
cfg['location']['depth'] = 200.0
cfg['time']['start'] = '2012-09-22 00:00:00'
cfg['time']['stop']  = '2012-12-05 03:00:00'
cfg['time']['dt']    = 600.0
cfg['grid']['nlev']  = nlev

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'hour'
cfg['output']['gotm_out']['time_step'] = 3
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = m.environ['gotmdir_data']+'/examples/OSMOSIS-Winter'
cfg['temperature']['method'] = 'file'
cfg['temperature']['file'] = datadir+'/t_prof.dat'
cfg['salinity']['method'] = 'file'
cfg['salinity']['file'] = datadir+'/s_prof.dat'
cfg['surface']['fluxes']['heat']['method'] = 'file'
cfg['surface']['fluxes']['heat']['file'] = datadir+'/heat_flux.dat'
cfg['surface']['fluxes']['tx']['method'] = 'file'
cfg['surface']['fluxes']['tx']['file'] = datadir+'/momentum_flux.dat'
cfg['surface']['fluxes']['tx']['column'] = 1
cfg['surface']['fluxes']['ty']['method'] = 'file'
cfg['surface']['fluxes']['ty']['file'] = datadir+'/momentum_flux.dat'
cfg['surface']['fluxes']['ty']['column'] = 2
cfg['surface']['swr']['method'] = 'file'
cfg['surface']['swr']['file'] = datadir+'/swr.dat'
cfg['surface']['precip']['method'] = 'file'
cfg['surface']['precip']['file'] = datadir+'/pme.dat'
cfg['waves']['stokes_drift']['us']['method'] = 'exponential'
cfg['waves']['stokes_drift']['vs']['method'] = 'exponential'
cfg['waves']['stokes_drift']['exponential']['us0']['method'] = 'file'
cfg['waves']['stokes_drift']['exponential']['us0']['file'] = datadir+'/stokes_drift.dat'
cfg['waves']['stokes_drift']['exponential']['us0']['column'] = 1
cfg['waves']['stokes_drift']['exponential']['vs0']['method'] = 'file'
cfg['waves']['stokes_drift']['exponential']['vs0']['file'] = datadir+'/stokes_drift.dat'
cfg['waves']['stokes_drift']['exponential']['vs0']['column'] = 2
cfg['waves']['stokes_drift']['exponential']['ds']['method'] = 'file'
cfg['waves']['stokes_drift']['exponential']['ds']['file'] = datadir+'/stokes_drift.dat'
cfg['waves']['stokes_drift']['exponential']['ds']['column'] = 3

# water type
cfg['light_extinction']['method'] = 'custom'
cfg['light_extinction']['A']['method'] = 'constant'
cfg['light_extinction']['A']['constant_value'] = 0.57
cfg['light_extinction']['g1']['method'] = 'constant'
cfg['light_extinction']['g1']['constant_value'] = 0.5
cfg['light_extinction']['g2']['method'] = 'constant'
cfg['light_extinction']['g2']['constant_value'] = 13.4

# EOS -- use linear
cfg['equation_of_state']['method'] = 'linear_custom'
cfg['equation_of_state']['linear']['T0'] = 10.0
cfg['equation_of_state']['linear']['S0'] = 35.0
cfg['equation_of_state']['linear']['alpha'] = 1.7e-4
cfg['equation_of_state']['linear']['beta'] = 7.8e-4

# damping on velocity: relaxation to zero with a 5-day decay time
cfg['velocities']['relax']['tau'] = 432000.0

Set the turbulence method to the generic length scale (GLS; [Umlauf and Burchard, 2003](https://doi.org/10.1357/002224003322005087)) model in the $k$-$\epsilon$ formulation with the weak-equilibrium stability function by [Canuto et al., 2001](https://doi.org/10.1175/1520-0485(2001)031%3C1413:OTPIOP%3E2.0.CO;2) (C01A).

In [ ]:
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3

## Run the model

In [ ]:
%%time
sim = m.run(config=cfg, label='SMC-C01A')

## Results

Load the data into an `xarray.Dataset`.

In [ ]:
data = sim.load_data()

Temperature

In [ ]:
fig = plt.figure(figsize=[8,4])
levels = np.linspace(12, 17, 51)
data.temp.plot(levels=levels)

## Try different turbulence models

Now run the same model with [CVMix](http://cvmix.github.io). Try three options in CVMix:

- KPP-CVMix ([Large et al., 1994](https://doi.org/10.1029/94RG01872), [Griffies et al., 2015](https://github.com/CVMix/CVMix-description/raw/master/cvmix.pdf))
- KPPLT-VR12 ([Li et al., 2016](https://doi.org/10.1016%2Fj.ocemod.2015.07.020))
- KPPLT-LF17 ([Li and Fox-Kemper, 2017](https://doi.org/10.1175%2FJPO-D-17-0085.1))

In [ ]:
cfgs = []
labels = []

In [ ]:
cfg['turbulence']['turb_method'] = 'cvmix'
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
cfgs.append(copy.deepcopy(cfg))
labels.append('KPP-CVMix')

In [ ]:
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'lwf16'
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-VR12')

In [ ]:
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'lf17'
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-LF17')

Run the three cases in parallel with 3 processes 

In [ ]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=3)

Plot the temperature in `KPP-CVMix` and the differences from it in `KPPLT-VR12` and `KPPLT-LF17`

In [ ]:
fig, axarr = plt.subplots(3, sharex='col')
fig.set_size_inches(8, 9)
data0 = sims[0].load_data()
levels = np.linspace(12, 17, 51)
data0.temp.plot(ax=axarr[0], levels=levels)
axarr[0].set_title(labels[0])
levels_diff = np.linspace(-2, 2, 41)
for i in np.arange(2):
    j = i+1
    diff = sims[j].load_data().temp - data0.temp
    diff.attrs['long_name'] = '$\Delta$ '+ data0.temp.attrs['long_name']
    diff.attrs['units'] = data0.temp.attrs['units']
    diff.plot(ax=axarr[j], levels=levels_diff)
    axarr[j].set_title(labels[j]+' $-$ '+labels[0])